In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

#### Загружаем очищенные данные

In [2]:
data19=pd.read_csv('df2019.csv',index_col=0)
data20=pd.read_csv('df2020.csv',index_col=0)
data21=pd.read_csv('df2021.csv',index_col=0)

data19.head(1)

,Наименование ДП,Нематериальные активы,Основные средства,Внеоборотные активы,Дебиторская задолженность,Оборотные активы,Уставный капитал,Капитал и резервы,Заёмные средства (долгосрочные),Долгосрочные обязательства,Заёмные средства (краткосрочные),Кредиторская задолженность,Краткосрочные обязательства,Выручка,Себестоимость продаж,Прибыль (убыток) до налогообложения,Прибыль (убыток) от продажи,"Макс. ПДЗ за 2019 год, дней"
0,0,9.050955e+06,2.145239e+09,2.440078e+09,3.475461e+09,6.077448e+09,629400000.0,2.289274e+09,0.0,2.712210e+08,3.923917e+08,5.476906e+09,5.957031e+09,3.703952e+10,3.438313e+10,7.461828e+08,8.716191e+08,0


#### Добавляем нужные столбцы

In [3]:
def other_columns(data):
    data['Активы']=data['Оборотные активы']+data['Внеоборотные активы']
    data['Собственные оборотные средства']=data['Оборотные активы']-data['Краткосрочные обязательства']
    data['Заёмные средства']=data['Заёмные средства (краткосрочные)']+data['Заёмные средства (долгосрочные)']
    data['Чистые активы']=data['Активы']-data['Заёмные средства']
    return data

data19=other_columns(data19)
data20=other_columns(data20)
data21=other_columns(data21)

#### Оставляем те компании, у которых активы не равны нулю

In [4]:
data19=data19[data19['Активы']!=0].reset_index().drop('index',axis=1)
data20=data20[data20['Активы']!=0].reset_index().drop('index',axis=1)
data21=data21[data21['Активы']!=0].reset_index().drop('index',axis=1)

#### Генерируем признаки

Подробнее: https://1fin.ru/?id=311

In [5]:
def feature_generation(data):
    # Создаём новый датафрейм
    df_new=pd.DataFrame()
    # Заполняем ключ
    df_new['ID']=data['Наименование ДП']
    # Генерим новые признаки
    print('Поехали, chanda belin')
    ### 1. Коэффициенты ликвидности
    df_new['cf_likv1']=data['Оборотные активы']/data['Краткосрочные обязательства'] # Коэффициент текущей ликвидности
    df_new['cf_likv2']=data['Оборотные активы']/(data['Краткосрочные обязательства']+data['Долгосрочные обязательства']) # Коэффициент общей ликвидности
    df_new['cf_likv3']=data['Оборотные активы']/(data['Краткосрочные обязательства']+data['Кредиторская задолженность']) # Общий коэффициент покрытия
    ### 2. Коэффициенты оборачиваемости
    df_new['cf_ob1']=data['Выручка']/data['Оборотные активы'] # Коэффициент оборачиваемости оборотных средств
    df_new['cf_ob2']=data['Выручка']/data['Дебиторская задолженность'] # Коэффициент оборачиваемости дебиторской задолженности
    df_new['cf_ob3']=data['Выручка']/data['Кредиторская задолженность'] # Коэффициент оборачиваемости кредиторской задолженности
    df_new['cf_ob4']=data['Выручка']/data['Капитал и резервы'] # Коэффициент оборачиваемости собственного капитала                    
    df_new['cf_ob5']=data['Оборотные активы']/data['Активы'] # Доля оборотных средств в активах
    ### 3. Коэффициенты рентабельности
    df_new['cf_rent1']=data['Прибыль (убыток) от продажи']/data['Выручка'] # Рентабельность продаж
    df_new['cf_rent2']=data['Прибыль (убыток) до налогообложения']/data['Себестоимость продаж'] # Рентабельность затрат
    df_new['cf_rent3']=data['Прибыль (убыток) до налогообложения']/data['Активы'] # Экономическая рентабельность  
    df_new['сf_rent4']=data['Прибыль (убыток) до налогообложения']/data['Капитал и резервы'] # Рентабельность собственного капитала
    df_new['сf_rent5']=data['Прибыль (убыток) до налогообложения']/data['Основные средства'] # Рентабельность основных средств
    df_new['сf_rent6']=data['Прибыль (убыток) до налогообложения']/(
        data['Заёмные средства (долгосрочные)']+data['Заёмные средства (краткосрочные)']) # Рентабельность заёмного капитала
    df_new['cf_rent7']=data['Прибыль (убыток) до налогообложения']/data['Оборотные активы'] # Рентабельность оборотных средств
    ### 4. Коэффициенты рыночной устойчивости
    df_new['cf_ru1']=data['Собственные оборотные средства']/data['Капитал и резервы'] # Коэффициент маневренности
    df_new['cf_ru2']=(data['Краткосрочные обязательства']+data['Долгосрочные обязательства'])/(
                      data['Капитал и резервы'])                                      # Коэффициент капитализации
    df_new['cf_ru3']=data['Собственные оборотные средства']/data['Оборотные активы'] # Коэффициент обеспеченности собственными средствами
    df_new['cf_ru4']=data['Капитал и резервы']/data['Заёмные средства'] # Коэффициент финансирования
    df_new['cf_ru5']=data['Дебиторская задолженность']/data['Кредиторская задолженность'] # Коэффициент дебиторской/кредиторской задолженности
    df_new['cf_ru6']=data['Краткосрочные обязательства']/(
                     data['Краткосрочные обязательства']+data['Долгосрочные обязательства']) # Коэффициент краткосрочной задолженности
    df_new['cf_ru7']=data['Долгосрочные обязательства']/data['Внеоборотные активы'] # Коэффициент структуры долгосрочных вложений
    df_new['cf_ru8']=data['Долгосрочные обязательства']/(
                    data['Долгосрочные обязательства']+data['Капитал и резервы']) # Коэффициент привлечения средств
    ### 5. Коэффициенты финансовой устойчивости
    df_new['cf_fu1']=(data['Капитал и резервы']+data['Заёмные средства (долгосрочные)'])/data['Активы'] # Коэффициент финансовой устойчивости
    df_new['cf_fu2']=data['Капитал и резервы']/data['Активы'] # Коэффициент финансовой независимости
    df_new['cf_fu3']=data['Заёмные средства']/data['Капитал и резервы'] # Коэффициент соотношения заемных и собственных средств
    df_new['cf_fu4']=data['Заёмные средства']/data['Активы'] # Коэффициент концентрации заемного капитала
    df_new['cf_fu5']=(data['Капитал и резервы']-data['Внеоборотные активы'])/data['Оборотные активы'] # Коэффициент обеспеченности оборотных активов собственными оборотными средствами
    df_new['cf_fu6']=data['Собственные оборотные средства']/data['Оборотные активы'] # Коэффициент обеспеченности собственными источниками финансирования
    
    return df_new

data19_new=feature_generation(data19)
data20_new=feature_generation(data20)
data21_new=feature_generation(data21)

Поехали, chanda belin
Поехали, chanda belin
Поехали, chanda belin


#### Добавляем таргет

In [6]:
data19_new['ПДЗ']=data19['Макс. ПДЗ за 2019 год, дней']
data20_new['ПДЗ']=data20['Макс. ПДЗ за 2020 год, дней']

#### Преобразуем таргет

In [7]:
def make_target_multi(data):
    data['target'] = np.where(data['ПДЗ']==0,0,
                                  np.where((data['ПДЗ']>0)&(data['ПДЗ']<=30),1,
                                  np.where((data['ПДЗ']>30)&(data['ПДЗ']<=60),2,
                                  np.where((data['ПДЗ']>60)&(data['ПДЗ']<=90),3,4))))
    
    return data

data19_new=make_target_multi(data19_new)
data20_new=make_target_multi(data20_new)

#### Преобразуем пропуски и бесконечности

In [8]:
def replace_na_inf(data):
    data=data.replace(np.inf,100)
    data=data.replace(-np.inf,100)
    data=data.fillna(0)
    return(data)
 
data19_new=replace_na_inf(data19_new)
data20_new=replace_na_inf(data20_new)
data21_new=replace_na_inf(data21_new)

#### Сохраняем в csv

In [9]:
data19_new.to_csv('df2019_model.csv')
data20_new.to_csv('df2020_model.csv')
data21_new.to_csv('df2021_model.csv')